In [1]:
from preamble import *

In [2]:
file_path = './data/d_pressure_flux_outflow.csv'

In [4]:
data = pd.read_csv(file_path)
data['datetime'] = pd.to_datetime(data['datetime'])
data_one_day = data[:1440]

In [5]:
import pickle

with open('best_polynomial_regression_model.pkl', 'rb') as f:
  model, poly = pickle.load(f)

c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PolynomialFeatures from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
from datetime import datetime, time

# 전기 요금 정보 (인상된 요금 반영)
electricity_rates = {
    'summer': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 92.46,  # 중간부하 요금 (원/kWh)
        'on_peak': 123.88   # 최대부하 요금 (원/kWh)
    },
    'spring_fall': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 69.50,  # 중간부하 요금 (원/kWh)
        'on_peak': 86.88    # 최대부하 요금 (원/kWh)
    },
    'winter': {
        'off_peak': 71.88,  # 경부하 요금 (원/kWh)
        'mid_peak': 90.80,  # 중간부하 요금 (원/kWh)
        'on_peak': 116.47   # 최대부하 요금 (원/kWh)
    }
}

# 시간대 구분
time_periods = {
    'summer': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'spring_fall': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'winter': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(10, 0)), (time(12, 0), time(17, 0)), (time(20, 0), time(22, 0))],
        'on_peak': [(time(10, 0), time(12, 0)), (time(17, 0), time(20, 0)), (time(22, 0), time(23, 0))]
    }
}

In [7]:
def get_season_and_period(dt):
  month = dt.month
  current_time = dt.time()
  
  if month in [7, 8]:
    season = 'summer'
  elif month in [11, 12, 1, 2]:
    season = 'winter'
  else:
    season = 'spring_fall'
  
  for period, times in time_periods[season].items():
    for start, end in times:
      if start <= current_time < end:
        return season, period
  return season, 'off_peak'

In [10]:
from sklearn.preprocessing import PolynomialFeatures

def calculate_daily_cost_by_linear(data):
  # data = pd.read_csv(file_path, parse_dates=['datetime'])
  total_cost = 0
  poly = PolynomialFeatures(3)
  for _, row in data.iterrows():
    season, period = get_season_and_period(row['datetime'])
    flux_poly = poly.fit_transform(np.array([[row['flux']]]))
    power_kW = model.predict(flux_poly)[0]
    cost_per_kWh = electricity_rates[season][period]
    energy_kWh = power_kW / 60  # converting kW to kWh per minute
    total_cost += energy_kWh * cost_per_kWh
    # print(power_kW)
  return total_cost

In [11]:
cost = calculate_daily_cost_by_linear(data_one_day)
cost_format = format(int(cost), ',')

In [12]:
from datetime import timedelta

def optimize_pump_flow(data, v_initial, capacity, max_flow = 250):
  outflow = data['outflow'].values
  minutes = len(data['datetime'])
  pump_flow = np.zeros(minutes)  # 각 분당 펌프 유량 설정
  over_flow = np.zeros(minutes)  # 넘으면 저장
  lower_flow = np.zeros(minutes)  # 모자라면 저장
  storage = v_initial  # 초기 배수지 저장량
  start_time = data['datetime'][0]
  v_min, v_max = capacity * 0.31, capacity * 0.94 # 1% 보수적 한계
  
  hourly_flow = 0  # 현재 1시간 동안 유지될 유량

  for minute in range(minutes):
    if minute % 60 == 0:  # 정각마다 유량 조정
      start_idx = minute
      end_idx = start_idx + 60
      expected_outflow = np.sum(outflow[start_idx:end_idx]) / 60  # 시간 단위로 변환
      
      # 전기 요금이 싼 심야 시간 우선 활용
      t = minute // 60 
      current_time = start_time + timedelta(hours=t)
      season, period = get_season_and_period(current_time)

      # cost_per_kWh = electricity_rates[season][period]
      
      # 저장량을 유지하기 위한 기본 필요 유입량
      hourly_flow = expected_outflow
      
      # 심야 시간 요금 절약을 위해 조정 (추가적인 충전 고려)
      if period == 'off_peak' and storage + hourly_flow - expected_outflow <= v_max:
        hourly_flow += max((v_max - (storage - expected_outflow)) * 0.3, 0) # 추가 충전
      elif period == 'mid_peak' and storage + hourly_flow - expected_outflow <= v_max:
        hourly_flow += max((v_max - (storage - expected_outflow)) * 0.15, 0) # 추가 충전
      if period == 'on_peak' and storage + hourly_flow - expected_outflow >= v_min:
        hourly_flow -= max(((storage - expected_outflow) - v_min) * 0.3, 0) # 절감 충전

      hourly_flow = max(hourly_flow, 0)
      hourly_flow = min(hourly_flow, max_flow)

    pump_flow[minute] = max(hourly_flow, 0)
    
    # 배수지에 반영
    storage += pump_flow[minute] - outflow[minute]
    if (storage > v_max):
      over_flow[minute] = storage - v_max
    elif (storage < v_min):
      lower_flow[minute] = v_min - storage

  over_non_zero_indices =  np.nonzero(over_flow)
  lower_non_zero_indices = np.nonzero(lower_flow)

  hourly_over_amount = np.zeros(24)
  hourly_lower_amount = np.zeros(24)
  
  if len(over_non_zero_indices[0]) != 0:
    for idx in over_non_zero_indices[0]:
      # print(f'{idx}분 over')
      h = idx // 60
      if hourly_over_amount[h] > (-1) * over_flow[idx]:
        hourly_over_amount[h] = (-1) * over_flow[idx]
  if len(lower_non_zero_indices[0]) != 0:
    for idx in lower_non_zero_indices[0]:
      # print(f'{idx}분 lower')
      h = idx // 60
      if hourly_lower_amount[h] < lower_flow[idx]:
        hourly_lower_amount[h] = lower_flow[idx]

  adjustment = hourly_over_amount + hourly_lower_amount
  # print(adjustment)

  for i, value in enumerate(adjustment):
    if value == 0: continue
    else:
      pump_flow[i * 60 : (i + 1) * 60] += (value / 60)

  return pump_flow

v_initial = 1419
capacity = 2000
optimized_flow = optimize_pump_flow(data_one_day, v_initial, capacity)

In [13]:
data_opti = data_one_day.copy()
data_opti['flux'] = optimized_flow
cost_opti = calculate_daily_cost_by_linear(data_opti)
cost_format_opti = format(int(cost_opti), ',')

In [14]:
saved_cost = cost - cost_opti
saved_cost_format = format(int(saved_cost), ',')
print(f'Total daily electricity cost: {cost_format} KRW')
print(f'Total daily electricity cost used optimization : {cost_format_opti} KRW')
print(f'Save money on electricity bills : {saved_cost_format} KRW')
print(f'electricity bill savings rate : {(1 - (int(cost_opti) / int(cost))) * 100:.2f} % 감소')

Total daily electricity cost: 352,454 KRW
Total daily electricity cost used optimization : 311,935 KRW
Save money on electricity bills : 40,518 KRW
electricity bill savings rate : 11.50 % 감소
